In [3]:
from google.colab import files

# Upload files
uploaded = files.upload()

import zipfile
import io

# Extracting the zip file
zip_ref = zipfile.ZipFile(io.BytesIO(uploaded['images.zip']), 'r')
zip_ref.extractall('/content') # or another directory where you want to extract
zip_ref.close()

Saving images.zip to images.zip


In [4]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Directory where the images are stored
base_dir = '/content/images'

# Image data generator with some data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

# Flow training images in batches
train_generator = train_datagen.flow_from_directory(base_dir, target_size=(150, 150), batch_size=32, class_mode='categorical')

# CNN 모델
model = tf.keras.models.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# 훈련
history = model.fit(train_generator, epochs=20)


Found 2347 images belonging to 804 classes.
Epoch 1/20
74/74 [==============================] - 32s 253ms/step - loss: 6.6964 - accuracy: 4.2608e-04
Epoch 2/20
74/74 [==============================] - 17s 227ms/step - loss: 6.6597 - accuracy: 0.0021
Epoch 3/20
74/74 [==============================] - 17s 230ms/step - loss: 6.4406 - accuracy: 0.0030
Epoch 4/20
74/74 [==============================] - 17s 230ms/step - loss: 6.1961 - accuracy: 0.0055
Epoch 5/20
74/74 [==============================] - 17s 226ms/step - loss: 6.0191 - accuracy: 0.0089
Epoch 6/20
74/74 [==============================] - 17s 230ms/step - loss: 5.8225 - accuracy: 0.0111
Epoch 7/20
74/74 [==============================] - 17s 232ms/step - loss: 5.6076 - accuracy: 0.0183
Epoch 8/20
74/74 [==============================] - 17s 227ms/step - loss: 5.3853 - accuracy: 0.0209
Epoch 9/20
74/74 [==============================] - 17s 229ms/step - loss: 5.1927 - accuracy: 0.0247
Epoch 10/20
74/74 [========================

In [5]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 테스트 이미지 업로드
uploaded_test_images = files.upload()

# 업로드한 이미지 파일명 가져오기
test_image_name = list(uploaded_test_images.keys())[0]  # 첫 번째 업로드한 이미지 이름

# 이미지 로드 및 사이즈 조정
test_image = load_img(test_image_name, target_size=(150, 150))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# 클래스 예측
result = model.predict(test_image)
predicted_class = np.argmax(result, axis=1)

# 해당 클래스의 레이블 가져오기
label_map = train_generator.class_indices
predicted_label = list(label_map.keys())[list(label_map.values()).index(predicted_class)]


Saving test.png to test.png
1/1 [==============================] - 0s 192ms/step


In [6]:
import pandas as pd
# 포켓몬 CSV 파일 업로드 및 정보 추출
uploaded_csv = files.upload()
pokemon_df = pd.read_csv('pokemon.csv')
filtered_df = pokemon_df[pokemon_df['Name'].str.lower() == predicted_label.lower()]

Saving pokemon.csv to pokemon.csv


In [7]:
# 포켓몬 타입 정보 가져오기
pokemon_info = pokemon_df[pokemon_df['Name'].str.lower() == predicted_label.lower()].iloc[0]

# 예측 확률 계산
predicted_probability = np.max(result) * 100

# Type1 및 Type2 정보 가져오기 (없는 경우 'None'으로 처리)
type1 = pokemon_info['Type1'] if not pd.isna(pokemon_info['Type1']) else 'None'
type2 = pokemon_info['Type2'] if not pd.isna(pokemon_info['Type2']) else 'None'

# 결과 출력
print(f"\"{predicted_label}\"이고, Type1은 \"{type1}\"이고, Type2는 \"{type2}\"입니다.")


"Fennekin"이고, Type1은 "Fire"이고, Type2는 "None"입니다.
